<a href="https://colab.research.google.com/github/Muyikem21/Sunday_INFO5502_Spring2022/blob/main/Lab_6_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The sixth Lab-assignment (03/24/2022, 50 points in total)

The purpose of this exercise is to build a simple predicition model which can helpyou understand the workflow of machine learning. 

### Task Decription 

The goal of this assignment is to predict bike share use, given the hour, day, and information about the weather. Companies like Divvy try to predict how much demand there will be for bikes on any given day to allocate resources to redistribute bikes so that, ideally, very few bike stations are ever full (when you can’t park your bike) or empty (when you can’t pick up a bike if you want to).

The data (link: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/lab_assignment_06) in Github provides detailed information on the data set and necessary downloads.

### Data Description

You are provided hourly rental data spanning two years (link: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/lab_assignment_06). For this task, the training set is comprised of the first 16 days of each month, while the test set is the 17-19th day of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period. That is, predict "count" without using "count" or its components "casual" and "registered".

Data Fields

datetime - hourly date + timestamp

season - 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather -

1 - Clear, Few clouds, Partly cloudy, Partly cloudy

2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals

### Output

Submission Format

Your output (a separate file) must have a header line and should be structured in the following format:

 datetime,count
 
 2011-01-20 00:00:00,0 
 
 2011-01-20 01:00:00,0
 
 2011-01-20 02:00:00,0
 
 ...
 
The tutorial code should demonstrate how to generate such a file from a very simple prediction model. Note, these prediction are to be done on the test file under the data tab, where you do not know the actual count, and should match the rows of the test file in count and order.

Your predictions should be compared to the ground truth information (sample_prediction.csv). Score are calculated using Root Mean Squared Error (RMSE).

### Tips

●	Add features: Pick columns/features from the data you already have. Or make a new feature from the ones you have. For example, the tutorial makes ‘hour’ from the datetime stamp, which seemed very useful. How about ‘month’?

●	Model selection: Try different models. Make sure they are regression models rather than classification models. Tip: random forest regression is a good, all around modeling strategy on complicated data sets.

●	Model tuning: Almost all regression models have parameters to tune (“hyperparameters”). E.g. polynomial regression has the degree of the polynomial (n = 1 for a line, n=2 for a quadratic fit, n=3 for a cubic fit…). Generally, one extreme makes the model too simple (e.g. a line for a curved set of points) and the other extreme makes the model overfit/be too complex, and usually the right choice is in between. For some models it is obvious what to tune (e.g. k for k nearest neighbors regression) and some don’t need much tuning with defaults that often work well. e.g. try changing the number of trees used in the random forest model!

●	Cross validation: The tutorial has a simple way of separating training and test data, however, there are better ways of splitting training and test data. Look into cross validation techniques, which are more reliable than an arbitrary split of training and test data.

●	Separate models for ...: Notice that count comes from just adding casual riders and registered riders. However, what if these two types of riders acted very differently? It might make sense to make two separate models and just add the results of both models together. This is also true for any subsets of your data that may behave wildly differently (e.g. create a separate model for each season?)

### Your code

In [1]:

import pandas as pd
import numpy as np
import csv as csv
from datetime import datetime
import matplotlib.pyplot as plt
from numpy.random import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

get_ipython().run_line_magic('matplotlib', 'inline')


In [3]:
# DATA READ
TRAINDATASET = pd.read_csv('train_luc.csv', header=0)
print("\nNumber of samples:", TRAINDATASET.shape[0], "and number of features:", TRAINDATASET.shape[1], "\n")

TRAINDATASET.head()



Number of samples: 9174 and number of features: 12 



,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:

#CONVERTING DATETIME TO DATE AND TIME
df = pd.DataFrame({'datetime':pd.date_range('2011-01-01 00:00:00',periods=9174)})
print("DataFrame is:\n", df)
TRAINDATASET['date'] = pd.to_datetime(df['datetime']).dt.date
TRAINDATASET['time'] = pd.to_datetime(df['datetime']).dt.time
print("Date-time-hour-minutes :\n", df)



DataFrame is:
        datetime
0    2011-01-01
1    2011-01-02
2    2011-01-03
3    2011-01-04
4    2011-01-05
...         ...
9169 2036-02-08
9170 2036-02-09
9171 2036-02-10
9172 2036-02-11
9173 2036-02-12

[9174 rows x 1 columns]
Date-time-hour-minutes :
        datetime
0    2011-01-01
1    2011-01-02
2    2011-01-03
3    2011-01-04
4    2011-01-05
...         ...
9169 2036-02-08
9170 2036-02-09
9171 2036-02-10
9172 2036-02-11
9173 2036-02-12

[9174 rows x 1 columns]


In [5]:
#MODEL DEFINITION
MODELDF = ['time']
model = LinearRegression()
print("columns selected for later:", MODELDF)
print(model)


columns selected for later: ['time']
LinearRegression()


In [6]:
#TRAIN AND TEST DATASETS

n = len(TRAINDATASET)
TRAIND = 0.75
INDEX = np.array(range(n))
shuffle(INDEX)
split_point = int(n * TRAIND)
TRAINI = INDEX[0:split_point]
TESTI = INDEX[split_point:]

TRAINDFN = TRAINDATASET.iloc[TRAINI]
TESTDFN = TRAINDATASET.iloc[TESTI]

print("TRAINING SUBSET SAMPLE:", len(TRAINDFN))
print("TEST SUBSET SAMPLE:", len(TESTDFN))



TRAINING SUBSET SAMPLE: 6880
TEST SUBSET SAMPLE: 2294


In [ ]:
#MODEL FIT
model.fit(TRAINDFN[MODELDF], TRAINDFN['count'])
PREDC = model.predict(TESTDFN[MODELDF])
rms = np.sqrt(mean_squared_error(TESTDFN['count'], PREDC))
print("RMS error:",rms)

TEST_DF = pd.read_csv('test_luc.csv', header=0)
print("\nNumber of samples:", TEST_DF.shape[0], "and number of features:", TEST_DF.shape[1], "\n")
TEST_DF['hour'] = TEST_DF['datetime'].map(hour_of_day)
TEST_DF.head()
model.fit(TRAINDATASET[MODELDF], TRAINDATASET['count'])
PREDC = model.predict(TEST_DF[MODELDF])
TEST_DF['count'] = PREDC


TEST_DF = TEST_DF[['datetime', 'count']].to_csv('my_prediction.csv',
                                                index=False, header=True)
print("Prediction complete. Saved as my_prediction.csv")


In [ ]:
#PREDICTION
TEST_DF = pd.read_csv('test_luc.csv', header=0)
print("\nNumber of samples:", TEST_DF.shape[0], "and number of features:", TEST_DF.shape[1], "\n")
TEST_DF['hour'] = TEST_DF['datetime'].map(hour_of_day)
TEST_DF.head()

model.fit(TRAINDATASET[MODELDF], TRAINDATASET['count'])
PREDC = model.predict(TEST_DF[MODELDF])
TEST_DF['count'] = PREDC
TEST_DF = TEST_DF[['datetime', 'count']].to_csv('my_prediction.csv',
                                                index=False, header=True)
print("PREDICITONS ARE SAVED TO CSV FILE")